# MRCONSO COL DESCRIPTION

Col.	Description

CUI	Unique identifier for concept

LAT	Language of term

TS	Term status

LUI	Unique identifier for term

STT	String type

SUI	Unique identifier for string

ISPREF	Atom status - preferred (Y) or not (N) for this string within this concept

AUI	Unique identifier for atom - variable length field, 8 or 9 characters

SAUI	Source asserted atom identifier [optional]

SCUI	Source asserted concept identifier [optional]

SDUI	Source asserted descriptor identifier [optional]

SAB	Abbreviated source name (SAB). Maximum field length is 20 alphanumeric characters. Two source abbreviations are assigned:
Root Source Abbreviation (RSAB) — short form, no version information, for example, AI/RHEUM, 1993, has an RSAB of "AIR"
Versioned Source Abbreviation (VSAB) — includes version information, for example, AI/RHEUM, 1993, has an VSAB of "AIR93"
Official source names, RSABs, and VSABs are included on the UMLS Source Vocabulary Documentation page.

TTY	Abbreviation for term type in source vocabulary, for example PN (Metathesaurus Preferred Name) or CD (Clinical Drug). Possible values are listed on the Abbreviations Used in Data Elements page.

CODE	Most useful source asserted identifier (if the source vocabulary has more than one identifier), or a Metathesaurus-generated source entry identifier (if the source vocabulary has none)

STR	String

SRL	Source restriction level

SUPPRESS	Suppressible flag. Values = O, E, Y, or N
O: All obsolete content, whether they are obsolesced by the source or by NLM. These will include all atoms having obsolete TTYs, and other atoms becoming obsolete that have not acquired an obsolete TTY (e.g. RxNorm SCDs no longer associated with current drugs, LNC atoms derived from obsolete LNC concepts).
E: Non-obsolete content marked suppressible by an editor. These do not have a suppressible SAB/TTY combination.
Y: Non-obsolete content deemed suppressible during inversion. These can be determined by a specific SAB/TTY combination explicitly listed in MRRANK.
N: None of the above
Default suppressibility as determined by NLM (i.e., no changes at the Suppressibility tab in MetamorphoSys) should be used by most users, but may not be suitable in some specialized applications. See the MetamorphoSys Help page for information on how to change the SAB/TTY suppressibility to suit your requirements. NLM strongly recommends that users not alter editor-assigned suppressibility, and MetamorphoSys cannot be used for this purpose.

CVF	Content View Flag. Bit field used to flag rows included in Content View. This field is a varchar field to maximize the number of bits available for use.


In [1]:
import pandas as pd

mrsty_file = '/home/ytaille/data/resources/umls/2017AB/MRSTY.RRF'
sgr_file = '/home/ytaille/data/resources/umls/semantic_groups_2018.txt'
# mrconso_file = '/home/ytaille/data/resources/umls/2014AB/MRCONSO.RRF'

# from glob import glob
# n2c2_path = '/home/ytaille/data/resources/n2c2/train_norm/*.norm'
# full_n2c2_df = pd.concat(
#     [pd.read_csv(f, sep='\|\|', names=['mention_id', 'CUI', 'begin', 'end'], index_col=False) for f in glob(n2c2_path)]
# )

mrsty_cols = ['CUI', 'TUI', 'STN', 'STY', 'ATUI', 'CVF']
# mrconso_cols = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF']

# n2c2_df = pd.read_csv(n2c2_file, sep='\|\|', names=['mention_id', 'CUI', 'begin', 'end'], index_col=False)
mrsty_df = pd.read_csv(mrsty_file, sep='\|', names=mrsty_cols, index_col=False)
mrsty_df.set_index('CUI', inplace=True)
sgr_df = pd.read_csv(sgr_file, sep='\|', names=['abbrev', 'class', 'TUI', 'description'], index_col=False)
sgr_df.set_index('TUI', inplace=True)
# mrconso_df = pd.read_csv(mrconso_file, sep='\|', names=mrconso_cols, index_col=False)


/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/ipykernel_launcher.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/ipykernel_launcher.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [5]:
# read train_file_list.txt
# for each file: conv to brat, keep CUI in annotatornotes 
# Example:
#2	AnnotatorNotes T2	C1530575

split = 'train'

with open(f'/home/ytaille/data/resources/n2c2/{split}_file_list.txt', 'r') as f :
    file_names = [fn.strip('\n') for fn in f.readlines()]
    
fail_cpt = 0
success_cpt = 0

for fn in file_names:
    norm_path = f'/home/ytaille/data/resources/n2c2/{split}_norm/{fn}.norm'
    note_path = f'/home/ytaille/data/resources/n2c2/{split}_note/{fn}.txt'
    
    norm_df = pd.read_csv(norm_path, sep='\|\|', names=['mention_id', 'CUI', 'begin', 'end'], index_col=False)
    
    with open(note_path, 'r') as f:
        note_txt = f.read()
        
    ann_str = ''
    mention_id = 0
        
    for _, mention in norm_df.iterrows():
        try:
            TUI = mrsty_df.loc[mention.CUI].TUI
            ann_label = sgr_df.loc[TUI].abbrev
            
            if isinstance(ann_label, pd.Series):
                ann_label = ann_label[0]
                
            ann_str += f'T{mention_id}\t{ann_label} {mention.begin} {mention.end}\t{note_txt[mention.begin:mention.end]}\n'
            ann_str += f'#{mention_id}\tAnnotatorNotes T{mention_id}\t{mention.CUI}\n'
            mention_id += 1
            success_cpt += 1
        except:
            if mention.CUI != 'CUI-less':
                print("TUI:", TUI)
                print("MENTION:", mention)
                print("#########")
                fail_cpt += 1
            
    with open(f'/home/ytaille/data/resources/n2c2/brat_files/{split}/{fn}.ann', 'w') as f:
        f.write(ann_str)
    with open(f'/home/ytaille/data/resources/n2c2/brat_files/{split}/{fn}.txt', 'w') as f:
        f.write(note_txt)

print("number of fails: ", fail_cpt, ", number of successes: ", success_cpt)

/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


number of fails:  0 , number of successes:  6533
